# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

data = pd.read_csv('homepage_actions.csv')

print(data.info())

print(data.head())

# Explore the id column to check for anomalies
print("Number of unique IDs:", data['id'].nunique())
print("Number of unique viewers:", data[data['action'] == 'view']['id'].nunique())
print("Number of unique clickers:", data[data['action'] == 'click']['id'].nunique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None
                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
Number of unique IDs: 6328
Number of unique viewers: 6328
Number of unique clickers: 1860


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [4]:
import pandas as pd
from scipy.stats import chi2_contingency
data['count'] = 1

contingency_table = pd.crosstab(data['group'], data['action'])

print(contingency_table)

# Perform the chi-square test
chi2, p, _, _ = chi2_contingency(contingency_table)

# Print the test statistic and p-value
print(f"Chi-square test statistic: {chi2}")
print(f"P-value: {p}")

# Determine statistical significance
alpha = 0.05
if p < alpha:
    print("Reject the null hypothesis. There is a significant difference between groups.")
else:
    print("Fail to reject the null hypothesis. There is no significant difference between groups.")


action      click  view
group                  
control       932  3332
experiment    928  2996
Chi-square test statistic: 3.636160051233291
P-value: 0.056537191086915774
Fail to reject the null hypothesis. There is no significant difference between groups.


In [5]:
from statsmodels.stats.proportion import proportions_ztest
control_clicks = contingency_table.loc['control', 'click']
control_views = contingency_table.loc['control', 'view']

experiment_clicks = contingency_table.loc['experiment', 'click']
experiment_views = contingency_table.loc['experiment', 'view']

z_stat, p_value = proportions_ztest([control_clicks, experiment_clicks], [control_views, experiment_views])

print("Z-statistic:", z_stat)
print("P-value:", p_value)

Z-statistic: -2.618563885349469
P-value: 0.008830075576595804


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [6]:
control_click_rate = contingency_table.loc['control', 'click'] / contingency_table.loc['control'].sum()

expected_experiment_clicks = control_click_rate * contingency_table.loc['experiment'].sum()

print("Expected number of clicks for the experiment group:", expected_experiment_clicks)


Expected number of clicks for the experiment group: 857.6848030018762


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [10]:
observed_experiment_clicks = contingency_table.loc['experiment', 'click']
z_score = (observed_experiment_clicks - expected_experiment_clicks) / \
           np.sqrt(expected_experiment_clicks * (1 - control_click_rate))
print("Z-score:", z_score)

Z-score: 2.71607302278784


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [13]:
import scipy.stats as stats
p_value = 2 * stats.norm.cdf(-np.abs(z_score))

print("P-value:", p_value)

P-value: 0.006606134551853142


### Analysis:

Does this result roughly match that of the previous statistical test?


#### Key Findings

1. **Chi-square Test:**
   - Initial chi-square test showed a p-value of 0.0565, indicating no significant difference between groups.
   - Decision: Fail to reject the null hypothesis.

2. **Proportions Z-test:**
   - Proportions z-test revealed a p-value of 0.0088, suggesting a significant difference.
   - Decision: Reject the null hypothesis.

3. **Binomial Distribution & Z-score Analysis:**
   - Expected clicks for the experiment group were around 857.68.
   - The observed z-score was approximately 2.72.
   - Calculated p-value from the z-score was 0.0066.
   - Consistent with the proportions z-test, indicating statistical significance.

#### Interpretation

The consistent low p-values across different analyses suggest a meaningful improvement in user engagement with the experimental homepage. The experimental group, on average, exhibited a significantly higher click-through rate compared to the control group. This implies that the changes made to the homepage were effective in encouraging user clicks.

It's important to note that while statistical significance was observed, practical significance should also be considered. The improvements should align with the goals and objectives of the music app.

***Conclusion:***
The experimental homepage was found to be more effective than the control homepage, leading to a higher click-through rate. The evidence supports implementing the changes introduced in the experimental design.



## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.